In [1]:
from utils.image_shingle import ImageShingle
import os
import pandas as pd
import json
import utils.utils as utils

In [2]:
CRAWL_NAME = 'slate'
CRAWL_PATH = f'crawls/{CRAWL_NAME}/'

PATHS = []
with open(f"crawls/{CRAWL_NAME}/results.json") as log_file:
    results = json.load(log_file)
    for path in results:
        # if not results[path]["unknown_exception"]:
        PATHS.append(path)
        
PATHS = utils.get_directories(CRAWL_PATH)
print(len(utils.get_directories(CRAWL_PATH)))

1


In [3]:
def compare_clickstreams(baseline: str, comparison: str, name: str) -> pd.DataFrame:
    rows_list = []

    for path in PATHS:
        clickstreams = utils.get_directories(path)

        sample_size = 0

        total_clickstreams = 0
        website_sum = 0

        for clickstream in clickstreams:

            total_actions = 0
            clickstream_sum = 0

            for _ in range(10):
                all_cookies_path = f"{clickstream}/{baseline}-{total_actions}.png"
                no_cookies_path = f"{clickstream}/{comparison}-{total_actions}.png"
                
                if os.path.isfile(all_cookies_path) and os.path.isfile(no_cookies_path):
                    CHUNK_SIZE = 40  # Recommended by https://www.usenix.org/legacy/events/sec07/tech/full_papers/anderson/anderson.pdf
                    all_cookies_shingle = ImageShingle(all_cookies_path, CHUNK_SIZE)
                    no_cookies_shingle = ImageShingle(no_cookies_path, CHUNK_SIZE)

                    clickstream_sum += all_cookies_shingle.compare(no_cookies_shingle)
                    total_actions += 1

            sample_size += total_actions

            if total_actions != 0:
                clickstream_similarity = clickstream_sum / total_actions
                website_sum += clickstream_similarity

                total_clickstreams += 1
        
        if total_clickstreams != 0:
            website_similarity = website_sum / total_clickstreams
            website_difference = 1 - website_similarity

            website = os.path.basename(os.path.normpath(path))
            rows_list.append({
                "website": website,
                f"difference_{name}": website_difference,
                f"sample_size_{name}": sample_size
            })

    return pd.DataFrame(rows_list)

In [4]:
no_cookies = compare_clickstreams("baseline", "all_cookies", "control_group")
all_cookies = compare_clickstreams("baseline", "no_cookies", "remove_cookies")

In [5]:
merged_df = no_cookies.merge(all_cookies, on=["website"], how="inner")

In [11]:
def compare_with_control(baseline: str, control: str, experimental: str) -> pd.DataFrame:
    rows_list = []

    for path in PATHS:
        clickstreams = utils.get_directories(path)

        sample_size = 0

        total_clickstreams = 0
        website_sum = 0

        for clickstream in clickstreams:

            total_actions = 0
            clickstream_sum = 0

            for _ in range(10):
                baseline_path = f"{clickstream}/{baseline}-{total_actions}.png"
                control_path = f"{clickstream}/{control}-{total_actions}.png"
                experimental_path = f"{clickstream}/{experimental}-{total_actions}.png"
                
                if os.path.isfile(baseline_path) and os.path.isfile(control_path) and os.path.isfile(experimental_path):
                    CHUNK_SIZE = 40  # Recommended by https://www.usenix.org/legacy/events/sec07/tech/full_papers/anderson/anderson.pdf
                    baseline_shingle = ImageShingle(baseline_path, CHUNK_SIZE)
                    control_shingle = ImageShingle(control_path, CHUNK_SIZE)
                    experimental_shingle = ImageShingle(experimental_path, CHUNK_SIZE)

                    try:
                        clickstream_sum += ImageShingle.compare_with_control(baseline_shingle, control_shingle, experimental_shingle)
                    except:
                        print(baseline_path)
                    total_actions += 1

            sample_size += total_actions

            if total_actions != 0:
                clickstream_similarity = clickstream_sum / total_actions
                website_sum += clickstream_similarity

                total_clickstreams += 1
        
        if total_clickstreams != 0:
            website_similarity = website_sum / total_clickstreams
            website_difference = 1 - website_similarity

            website = os.path.basename(os.path.normpath(path))
            rows_list.append({
                "website": website,
                f"difference": website_difference,
                f"sample_size": sample_size
            })

    return pd.DataFrame(rows_list)

In [12]:
df = compare_with_control("baseline", "all_cookies", "no_cookies")
df

crawls/slate/slate.com/0/baseline-0.png
crawls/slate/slate.com/0/baseline-2.png


,website,difference,sample_size
0,slate.com,0.67533,3


In [8]:
# merged_df.to_csv(f'analysis/{CRAWL_NAME}.csv', index=False)

In [9]:
merged_df

,website,difference_control_group,sample_size_control_group,difference_remove_cookies,sample_size_remove_cookies
0,slate.com,0.378382,3,0.309027,3
